In [42]:
import pandas as pd

input_file_path = '/Users/rbnavaja/Documents/Safeway/pbs_automation/MNL Paygroup/Oncycle/2023_09_30/32959077/DONE_JSM_PQ0032CS__48_HOURS_018_33078472.xls'
input_df = pd.read_excel(input_file_path, sheet_name="Working", header=3)
display(input_df)

header_data_df = pd.read_excel(input_file_path, sheet_name="Working", usecols='A', skiprows=1, nrows=2, header=None)
print(header_data_df)

pay_period = header_data_df.iloc[0][0].split("=")[-1].strip()
pay_group = header_data_df.iloc[1][0].split("=")[-1].strip()
print(pay_period)
print(pay_group)

,Empl ID,Name,Earn Code,Sum Oth Hrs,Location,Sep Check Nbr
0,1025516,"Minium,Amber Dawn",65A,60.25,0513A,0
1,1038876,"Irey,James Herbert",065,56.60,6629A,0
2,1743228,"Smith,Thomas J",065,49.17,6629A,0
3,1777379,"Scott,Douglas S",065,55.15,6629A,0
4,1777548,"Schiermeister,Troy A",065,70.40,6629A,0
...,...,...,...,...,...,...
86,20368609,"Quintero,Danny arthur",065,48.88,6629A,0
87,20413641,"Newton,Carl Anthony",065,55.82,6629A,0
88,20413643,"Coburn,Ervin",065,54.25,6629A,0
89,20505073,"Erickson,Sean B",065,59.20,6629A,0


                                  0
0  Pay Period End Date = 2023-10-07
1                       Group = 018
2023-10-07
018


In [45]:
input_df['Pay Period'] = pay_period
input_df['Pay Group'] = pay_group
input_df

validation_data_df = input_df.loc[:, ["Empl ID", "Earn Code", 'Pay Period' , 'Pay Group']]
validation_data_df.to_dict('records')

[{'Empl ID': 1025516,
  'Earn Code': '65A',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1038876,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1743228,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1777379,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1777548,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1795657,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1799271,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1823269,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1841989,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1845083,
  'Earn Code': '065',
  'Pay Period': '2023-10-07',
  'Pay Group': '018'},
 {'Empl ID': 1876140,
  'Earn 

In [2]:
import xlrd
from xlutils.copy import copy as xl_copy

# open existing workbook
rb = xlrd.open_workbook(input_file_path, formatting_info=True)
# make a copy of it
wb = xl_copy(rb)
# add sheet to workbook with existing sheets
ws = wb.get_sheet(0)
ws.name = 'test1'
Sheet1 = wb.add_sheet('Working2')
wb.save(input_file_path)

In [1]:
class VanillaFileInfo:
    folder_path = 'MNL Paygroup/Oncycle'
    file_substr_list = {
        'AM_PLA_PAYLINE_E',
        'MLA_AM_DETAIL_ADV_VAC',
        'HOURLY_EXEMPT_CHANGE_DLOWE'
    }
    base_sheet_name = "sheet1"
    copy_sheet_name = "Working"
    new_sheet_name = "Original"

class MaildropFileInfo:
    folder_path = 'MNL Paygroup/Oncycle'
    file_substr_list = {
        'PQ0032CS__48_HOURS',
        'PY_OVERPAYMENT_WAGE_ANALYSIS'
    }
    base_sheet_name = "sheet1"
    copy_sheet_name = "Working"
    new_sheet_name = "Original"


In [2]:
import os
from glob import glob
from dateutil.parser import parse
from datetime import datetime
from xlrd import open_workbook
from xlutils.copy import copy as xl_copy
import xlwings as xw

def validate_date_format(date_string, date_format='%Y_%m_%d'):
        try:
            is_valid_date = bool(datetime.strptime(date_string, date_format))
        except ValueError:
            is_valid_date = False

        return is_valid_date


def get_input_file_path(folder_path, file_substr):
    # print(folder_path, file_substr)

    sub_folder_list = os.listdir(folder_path)
    date_folder_list = [sub_folder for sub_folder in sub_folder_list if validate_date_format(sub_folder)]

    if not date_folder_list:
        raise Exception(f"No Valid Date Folder Found in {folder_path}")

    latest_date_folder = max(date_folder_list)
    latest_date_folder_path = os.path.join(folder_path, latest_date_folder, "")
    # print(latest_date_folder_path)

    matched_path_list = glob(f"{latest_date_folder_path}/**/*{file_substr}*", recursive=True)

    if matched_path_list:
        return matched_path_list[0]

info_class_list = [VanillaFileInfo, MaildropFileInfo]
for info_class in info_class_list:
    for file_substr in info_class.file_substr_list:
        input_file_path = get_input_file_path(info_class.folder_path, file_substr)
        print(input_file_path)
        app = xw.App(visible=False)
        work_book = xw.Book(input_file_path)
        sheet_names = work_book.sheet_names

        base_sheet_name = info_class.base_sheet_name
        copy_sheet_name = info_class.copy_sheet_name
        new_sheet_name = info_class.new_sheet_name
        if base_sheet_name in sheet_names and copy_sheet_name not in sheet_names:
            print(f"Sheets to Prepare: {sheet_names}")
            print(f"Duplicating {base_sheet_name} as {copy_sheet_name}")
            base_sheet = work_book.sheets[base_sheet_name]
            base_sheet.copy(name=copy_sheet_name)
            print(f"Renaming {base_sheet_name} to {new_sheet_name}")
            base_sheet.name = new_sheet_name
            work_book.save()

        sheet_names = work_book.sheet_names
        print(f"Sheets to Validate: {sheet_names}")

        work_book.app.quit()

MNL Paygroup/Oncycle/2023_09_30/32959499/DONE_MEB_MLA_AM_DETAIL_ADV_VAC_033_33301990.xls
Sheets to Validate: ['Original', 'WORKING1', 'Working']
MNL Paygroup/Oncycle/2023_09_30/32959499/DONE_RLM_AM_PLA_PAYLINE_E_033_32959499.xls
Sheets to Validate: ['Original', 'WORKING1', 'Working']
MNL Paygroup/Oncycle/2023_09_30/32959499/DONE_MEB_HOURLY_EXEMPT_CHANGE_DLOWE_033_32959499.xls
Sheets to Validate: ['Original', 'WORKING1', 'Working']
MNL Paygroup/Oncycle/2023_09_30/33079159/DONE_JSM_PQ0032CS__48_HOURS_018_33078472.xls
Sheets to Validate: ['Working', 'Original']
MNL Paygroup/Oncycle/2023_09_30/33079159/DONE_JSM_PY_OVERPAYMENT_WAGE_ANALYSIS_018_33078472.xls
Sheets to Validate: ['Working', 'Original']


In [10]:
import pandas as pd

input_df = pd.read_excel('MNL Paygroup/Oncycle/2023_09_30/32959499/DONE_MEB_MLA_AM_DETAIL_ADV_VAC_033_33301990.xls', sheet_name="Original", header=3) 
input_df 

,Group,Pay Period End,ID,Name,Service Dt,Labor Agreement,Sep Check Nbr,Earn Code,Oth Hrs,Earns Begin,Earns End,Stnd Hrs/Wk
0,33,2023-10-21,1179342,"Ventura,Scott T",1990-04-17,1151RI,1,321,8.0,2023-10-23,2023-10-23,40
1,33,2023-10-21,1179342,"Ventura,Scott T",1990-04-17,1151RI,1,321,8.0,2023-10-24,2023-10-24,40
2,33,2023-10-21,1179342,"Ventura,Scott T",1990-04-17,1151RI,1,321,8.0,2023-10-25,2023-10-25,40
3,33,2023-10-21,1179342,"Ventura,Scott T",1990-04-17,1151RI,1,321,8.0,2023-10-26,2023-10-26,40
4,33,2023-10-21,1179342,"Ventura,Scott T",1990-04-17,1151RI,1,321,8.0,2023-10-27,2023-10-27,40
...,...,...,...,...,...,...,...,...,...,...,...,...
469,33,2023-10-21,9970195,"Stann,Robert Michael",2015-01-14,1151MA,1,321,8.0,2023-10-23,2023-10-23,40
470,33,2023-10-21,9970195,"Stann,Robert Michael",2015-01-14,1151MA,1,321,8.0,2023-10-24,2023-10-24,40
471,33,2023-10-21,9970195,"Stann,Robert Michael",2015-01-14,1151MA,1,321,8.0,2023-10-25,2023-10-25,40
472,33,2023-10-21,9970195,"Stann,Robert Michael",2015-01-14,1151MA,1,321,8.0,2023-10-26,2023-10-26,40


In [11]:
to_filter_df = input_df.loc[:, ['ID', 'Sep Check Nbr', 'Oth Hrs', 'Stnd Hrs/Wk']]
to_filter_df

,ID,Sep Check Nbr,Oth Hrs,Stnd Hrs/Wk
0,1179342,1,8.0,40
1,1179342,1,8.0,40
2,1179342,1,8.0,40
3,1179342,1,8.0,40
4,1179342,1,8.0,40
...,...,...,...,...
469,9970195,1,8.0,40
470,9970195,1,8.0,40
471,9970195,1,8.0,40
472,9970195,1,8.0,40


In [12]:
aggregated_df = to_filter_df.groupby(['ID']).agg({'ID':'count', 'Sep Check Nbr': [pd.Series.nunique], 'Oth Hrs': 'sum', 'Stnd Hrs/Wk' : 'first'}).reset_index()
aggregated_df

ID       Sep Check Nbr Oth Hrs Stnd Hrs/Wk
             count       nunique     sum       first
0    1179342     5             1   40.00          40
1    1179547     5             1   40.00          40
2    1181036     5             1   40.00          40
3    1181757     5             1   40.00          40
4    1182751     5             1   40.00          40
..       ...   ...           ...     ...         ...
90  20182336     5             1   33.75          20
91  20189921     5             1   40.00          40
92  20197773     5             1   21.25          24
93  20198850     5             1   36.05          20
94  20201643     5             1   26.30          24

[95 rows x 5 columns]

In [13]:
aggregated_df.columns = aggregated_df.columns.map(' '.join).str.strip(' ').str.replace(' first', '')
aggregated_df

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk
0,1179342,5,1,40.00,40
1,1179547,5,1,40.00,40
2,1181036,5,1,40.00,40
3,1181757,5,1,40.00,40
4,1182751,5,1,40.00,40
...,...,...,...,...,...
90,20182336,5,1,33.75,20
91,20189921,5,1,40.00,40
92,20197773,5,1,21.25,24
93,20198850,5,1,36.05,20


In [14]:
aggregated_df.query("`ID count` < 5")

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk
83,6184684,4,1,32.0,40


In [15]:
aggregated_df.query("`ID count` > 5")

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk


In [16]:
aggregated_df['No Action'] = False
aggregated_df

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action
0,1179342,5,1,40.00,40,False
1,1179547,5,1,40.00,40,False
2,1181036,5,1,40.00,40,False
3,1181757,5,1,40.00,40,False
4,1182751,5,1,40.00,40,False
...,...,...,...,...,...,...
90,20182336,5,1,33.75,20,False
91,20189921,5,1,40.00,40,False
92,20197773,5,1,21.25,24,False
93,20198850,5,1,36.05,20,False


In [17]:
no_action_filter_1 = (aggregated_df['ID count'] == 5) & (aggregated_df['Oth Hrs sum'] >= aggregated_df['Stnd Hrs/Wk'])
display(aggregated_df[no_action_filter_1])
aggregated_df.loc[no_action_filter_1, ['No Action']] = True
aggregated_df

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action
0,1179342,5,1,40.00,40,False
1,1179547,5,1,40.00,40,False
2,1181036,5,1,40.00,40,False
3,1181757,5,1,40.00,40,False
4,1182751,5,1,40.00,40,False
...,...,...,...,...,...,...
89,20133070,5,1,25.75,15,False
90,20182336,5,1,33.75,20,False
91,20189921,5,1,40.00,40,False
93,20198850,5,1,36.05,20,False


,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action
0,1179342,5,1,40.00,40,True
1,1179547,5,1,40.00,40,True
2,1181036,5,1,40.00,40,True
3,1181757,5,1,40.00,40,True
4,1182751,5,1,40.00,40,True
...,...,...,...,...,...,...
90,20182336,5,1,33.75,20,True
91,20189921,5,1,40.00,40,True
92,20197773,5,1,21.25,24,False
93,20198850,5,1,36.05,20,True


In [18]:
no_action_filter_2 = (aggregated_df['ID count'] > 5) & (aggregated_df['Oth Hrs sum'] >= aggregated_df['Stnd Hrs/Wk']) & (aggregated_df['Sep Check Nbr nunique'] == 1)
display(aggregated_df[no_action_filter_2])
aggregated_df.loc[no_action_filter_2, ['No Action']] = True
aggregated_df

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action


,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action
0,1179342,5,1,40.00,40,True
1,1179547,5,1,40.00,40,True
2,1181036,5,1,40.00,40,True
3,1181757,5,1,40.00,40,True
4,1182751,5,1,40.00,40,True
...,...,...,...,...,...,...
90,20182336,5,1,33.75,20,True
91,20189921,5,1,40.00,40,True
92,20197773,5,1,21.25,24,False
93,20198850,5,1,36.05,20,True


In [19]:
no_action_filter_3 = (aggregated_df['ID count'] < 5) & (aggregated_df['Oth Hrs sum'] >= aggregated_df['Stnd Hrs/Wk'])
display(aggregated_df[no_action_filter_3])
aggregated_df.loc[no_action_filter_3, ['No Action']] = True
aggregated_df

,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action


,ID,ID count,Sep Check Nbr nunique,Oth Hrs sum,Stnd Hrs/Wk,No Action
0,1179342,5,1,40.00,40,True
1,1179547,5,1,40.00,40,True
2,1181036,5,1,40.00,40,True
3,1181757,5,1,40.00,40,True
4,1182751,5,1,40.00,40,True
...,...,...,...,...,...,...
90,20182336,5,1,33.75,20,True
91,20189921,5,1,40.00,40,True
92,20197773,5,1,21.25,24,False
93,20198850,5,1,36.05,20,True


In [ ]:
aggregated_df.query("~`No Action`")

what to do with id count equals to 5 but sep >